In [8]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd
import os

# MongoDB connection
from animal_crud import AnimalShelter

username = "accuser"
password = "SecurePassword123"
db = AnimalShelter(username, password)

# Initial full dataset
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Encode Grazioso Salvare logo
image_filename = 'my-image.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Start the Dash app
app = JupyterDash(__name__)

# Layout
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard — Liam O’Brien'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '100px'}),
    html.Hr(),

    html.Label("Select Outcome Subtype"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Aggressive', 'value': 'Aggressive'},
            {'label': 'Rabies Risk', 'value': 'Rabies Risk'},
            {'label': 'In Foster', 'value': 'In Foster'},
            {'label': 'Reset All Filters', 'value': 'Reset'}
        ],
        value='Reset',
        inline=True
    ),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        row_selectable='single',
        page_size=10,
        style_table={'overflowX': 'auto'}
    ),

    html.Br(), html.Hr(),

    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callback: Filter the table by outcome_subtype
@app.callback(Output('datatable-id', 'data'), Input('filter-type', 'value'))
def update_dashboard(filter_type):
    if filter_type == "Reset":
        data = db.read({})
    else:
        data = db.read({"outcome_subtype": {"$regex": filter_type, "$options": "i"}})

    df = pd.DataFrame.from_records(data)
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

# Callback: Pie chart of breed distribution
@app.callback(Output('graph-id', "children"), Input('datatable-id', "derived_virtual_data"))
def update_graph(view_data):
    if view_data:
        dff = pd.DataFrame.from_dict(view_data)
        if not dff.empty:
            fig = px.pie(dff, names='breed', title='Breed Distribution')
            return [dcc.Graph(figure=fig)]
    return [html.Div("No data available for chart")]

# Callback: Highlight selected column
@app.callback(Output('datatable-id', 'style_data_conditional'),
              Input('datatable-id', 'selected_columns'))
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback: Geolocation map of selected row
@app.callback(Output('map-id', "children"),
              [Input('datatable-id', "derived_virtual_data"),
               Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    if not viewData or not index:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return []

    row = index[0]
    try:
        lat = dff.iloc[row, 13]
        lon = dff.iloc[row, 14]
        breed = dff.iloc[row, 4]
        name = dff.iloc[row, 9]
    except:
        return []

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]

app.run_server(mode='inline')


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: 'NoneType' object is not iterable

